In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
import time
import random

In [11]:
plm=pd.read_excel("C:/Users/claud/Downloads/plm_limpio.xlsx")

In [12]:
#Eliminar duplicados de nombre_producto
plm=plm.drop_duplicates(subset=['nombre_producto'], keep='first')
plm

,nombre_producto,sustancia,forma_farm,lab,presentacion
0,a.f. valdecasas,vitamina b9 (acido folico),tabletas,valdecasas,"1 frasco(s) , 20 tabletas , 1.0 miligramos"
10,a.s. cor,norfenefrina,solucion,sanfer,"1 frasco con gotero, 24 ml,"
11,abacavir,abacavir,tabletas,cryopharma,"1 caja , 60 tabletas , 300 miligramos"
12,abatoarin,tropisetron,capsulas,pisa,"1 caja , 1 capsulas , 5 miligramos"
15,abbatin,montelukast,tabletas masticables,corne,"1 caja,20 tabletas masticables,5 mg"
...,...,...,...,...,...
16634,zyrtec,cetirizina,solucion,armstrong,"1 caja, 1 frasco con gotero, 10 ml, 10 mg/ml"
16648,zytiga,abiraterona,tabletas,janssen-cilag,"1 caja, 1 frasco(s), 120 tabletas, 250 miligramos"
16650,zytokil,doxorubicina,solucion inyectable,pisa,NaN
16651,zyvoxam,linezolid,solucion inyectable,pfizer,"1 sobre con bolsa de infusion, 300 ml, 2 mg/ml"


In [13]:
#Crear lista con nombre producto por grupos de 500
lista=plm['nombre_producto'].tolist()
#Hacer lista de listas separadas por 200
lista2=[lista[i:i + 400] for i in range(0, len(lista), 400)]

In [14]:

"""Esta función busca en la página de la COFEPRIS el registro sanitario de un medicamento
    
    """

def registro(medicamento):
    #función para cambiar la denominación del medicamento
    def denominacion(option_text):
        combo = driver.find_element(By.CLASS_NAME, "form-control")
        combo_select = Select(combo)
        combo_select.select_by_visible_text(option_text)
        cuadro_busqueda = driver.find_element(By.ID, "MainContent_txtSearchValue")
        cuadro_busqueda.send_keys(medicamento)
        button = driver.find_element(By.XPATH, "//button[contains(@onclick, '__doPostBack') and @class='btn btn-primary']")
        button.click()
    #Función para extraer los elementos de la tabla
    def elementos_tabla(driver):
        table_element = driver.find_element(By.CLASS_NAME, "gridView_basic")
        table_contents = table_element.get_attribute('outerHTML')
        return table_contents
    
    #Código para extraer los datos de la página
    url = "https://tramiteselectronicos02.cofepris.gob.mx/BuscadorPublicoRegistrosSanitarios/BusquedaRegistroSanitario.aspx"
    options = webdriver.ChromeOptions()
    options.add_argument('--incognito')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-cache')
    options.add_argument('--disable-cookies')
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service("/usr/local/bin/chromedriver"), options=options)
    driver.get(url)
    #Denominación Distintiva
    denominacion("Denominación Distintiva")

    try:
        table_contents = elementos_tabla(driver)
    except NoSuchElementException:
        #Denominación Genérica
        denominacion("Denominación Genérica")
        try:
            table_contents = elementos_tabla(driver)
        except NoSuchElementException:
            driver.quit()
            return pd.DataFrame()

    soup = BeautifulSoup(table_contents, 'html.parser')
    registro=[]
    marca = []
    generica = []
    tipo_med=[]
    indica = []
    lab = []
    fabricante = []
    principio=[]

    for span in soup.find_all('span'):
        if span.has_attr('id'):
            if span['id'].startswith('MainContent_GV_RegistroSanitario_lblNumeroRegistro'):
                registro.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionDistintiva'):
                marca.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblDenominacionGenerica'):
                generica.append(span.text)
            elif span["id"].startswith('MainContent_GV_RegistroSanitario_lblTipoMedicamento'):
                tipo_med.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblTitularRegistro'):
                lab.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblFabricanteMedicamento'):
                fabricante.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblPrincipioActivo'):
                principio.append(span.text)
            elif span['id'].startswith('MainContent_GV_RegistroSanitario_lblIndicacionTerapeutica'):
                indica.append(span.text)

    df = pd.DataFrame({'reg_san':registro,'marca': marca, 'den_gen': generica, 'tipo_med':tipo_med, 'titular': lab, 'fabricante': fabricante, 'principio': principio,'indicacion':indica,"busqueda":medicamento})
    df = df.apply(lambda x: x.astype(str).str.lower())
    df = df.apply(lambda x: x.astype(str).str.replace('\n', ''))
    driver.quit()
    return df

In [15]:

tabla = pd.DataFrame()

for i in lista2:
    success = True  # Flag variable to track if an exception occurred
    for j in i:
        try:
            # Búsqueda en una variable
            df = registro(j)
            if not df.empty:
                tabla = pd.concat([tabla, df])
            # Tiempo entre elementos
            time.sleep(random.randint(1, 2))
        except WebDriverException as e:
            if "no such element: Unable to locate element" in str(e):
                print(f"WebDriverException encontrado: {str(e)}")
                print("Medicamento: ", j)
                print("Saltando hacia el siguiente elemento...")
            else:
                print(f"WebDriverException encontrado: {str(e)}")
                print("Reintentando la búsqueda...")
            continue  # Skip to the next element in the inner loop

    # Tiempo entre listas
    time.sleep(random.randint(240, 300))

tabla


WebDriverException encontrado: Message: no such element: Unable to locate element: {"method":"css selector","selector":".form-control"}
  (Session info: headless chrome=114.0.5735.134); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0117A813+48355]
	(No symbol) [0x0110C4B1]
	(No symbol) [0x01015358]
	(No symbol) [0x010409A5]
	(No symbol) [0x01040B3B]
	(No symbol) [0x0106E232]
	(No symbol) [0x0105A784]
	(No symbol) [0x0106C922]
	(No symbol) [0x0105A536]
	(No symbol) [0x010382DC]
	(No symbol) [0x010393DD]
	GetHandleVerifier [0x013DAABD+2539405]
	GetHandleVerifier [0x0141A78F+2800735]
	GetHandleVerifier [0x0141456C+2775612]
	GetHandleVerifier [0x012051E0+616112]
	(No symbol) [0x01115F8C]
	(No symbol) [0x01112328]
	(No symbol) [0x0111240B]
	(No symbol) [0x01104FF7]
	BaseThreadInitThunk [0x76327D59+25]
	RtlInitializeExceptionChain [0x7760B74B+107]
	R

,reg_san,marca,den_gen,tipo_med,titular,fabricante,principio,indicacion,busqueda
0,82231 ssa,a.f. valdecasas,ácido fólico,de referencia,laboratorios valdecasas s.a.,laboratorios valdecasas s.a.,b03bb01 ácido fólico,antianémico y en mujeres de edad reproductiva ...,a.f. valdecasas
1,002v2000 ssa,a.f. valdecasas.,ácido fólico,de referencia,"laboratorios valdecasas, s. a.",laboratorios valdecasas s.a.,b03bb01 ácido fólico,"para las deficiencias del ácido fólico, preven...",a.f. valdecasas
2,045v2000 ssa,a.f. valdecasas,ácido fólico,de referencia,"laboratorios valdecasas, s. a.",laboratorios valdecasas s.a.,b03bb01 ácido fólico,para las deficiencias del ácido fólico.,a.f. valdecasas
0,136m2014 ssa,a.s. cor,norfenefrina,de referencia,"grimann, s. a. de c.v.","grimann, s.a. de c.v.",c01ca03 norepinefrina.,vasoconstrictor,a.s. cor
0,282m2018 ssa,narkson,abacavir / lamivudina,genérico,"nucitec, s.a. de c.v.","nucitec, s.a. de c.v.",,para el tratamiento de la infección causada po...,abacavir
...,...,...,...,...,...,...,...,...,...
0,099m2003 ssa,zytokil,doxorubicina,genérico,"laboratorios pisa, s.a. de c.v.","laboratorios pisa, s.a. de c.v.",,para el tratamiento de la leucemia aguda linfo...,zytokil
0,105m2000 ssa,zyvoxam,linezolid,de referencia,"pfizer, s. a. de c. v.",pfizer pharmaceuticals llc,j01xx08 linezolid,antibiótico (oxazolidinona) para infecciones p...,zyvoxam
1,159m2000 ssa,zyvoxam,linezolid,de referencia,"pfizer, s.a. de c.v.",fresenius kabi norge as,j01xx08 linezolid,antibiótico (oxazolidinona) para infecciones o...,zyvoxam
0,282m2016 ssa,zyxem,levocetirizina,genérico,"productos farmaceuticos, s.a. de c.v.",aesica pharmaceuticals s.r.l.,r06ae09 levocetirizina,antihistamínico,zyxem


In [17]:
#Salvar tabla en csv
tabla.to_csv("C:/Users/claud/Downloads/registro_sanitario.csv",index=False, encoding='utf-8-sig')

In [18]:
len(tabla["busqueda"].unique())

3236

In [21]:
len(tabla["busqueda"].unique())/len(lista)*100

60.15988101877673